In [ ]:
from load_docs import load_documents,chunk_documents
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFaceHub
import os
import streamlit as st
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFaceEndpoint
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# 1. Load PDF
documents = load_documents("docs/na_tesla_service_plan_agreement_ot.pdf")

In [ ]:
# 2. Split into chunks
chunks = chunk_documents(documents)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en")
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device=-1   # CPU, or 0 for GPU
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [ ]:
# 4. Create QA chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
# Point to your deployed Inference Endpoint
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever
)